In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import pearsonr as p

from sklearn.linear_model import LinearRegression as lr
from catboost import CatBoostRegressor as cbr

import sys
sys.path.append('../input/pycaret/pyod')
sys.path.append('../input/pycaret/pycaret')
from pycaret.regression import *

import warnings
warnings.simplefilter('ignore')

In [ ]:
def corr(y_pred, y_true):
    return p(y_pred, y_true)[0]

In [ ]:
train = pd.read_pickle('../input/ump195gb/train.pkl')
train.head(3)

In [ ]:
investments = train.investment_id.unique()

In [ ]:
for i in range(600,900):
    train_sub = train.groupby('investment_id').get_group(investments[i])
    
    reg = setup(data = train_sub.drop(['row_id', 'time_id', 'investment_id'], axis=1),
            target = 'target',
            train_size = 0.75, 
            normalize = True, 
            normalize_method = 'robust', 
            transform_target = False, 
            data_split_shuffle = False, 
            feature_interaction = False, 
            silent = True, 
            fold = 5, 
            n_jobs = -1) 
    
    add_metric('CORR', 'CORR', corr, greater_is_better=True)
    
    #model = compare_models(sort='CORR', n_select=1, exclude=['xgboost', 'catboost', 'rf'])
    model = create_model('dt')
    model = tune_model(model, n_iter=200, optimize='CORR')
    
    save_model(model, f'model_investment_{investments[i]}')
    print('_'*50)
    print(f'Investment: {investments[i]}')
    print(model)